In [2]:
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import grad

# device = torch.device('cuda')
device = torch.device('cpu')

sys.path.append('utils_continuous')
from utils_continuous import preprocess_data_continuous_identification, plot_results_continuous_identification

In [4]:
# Load the dataset and preprocess it
data_path = 'data/burgers_shock.mat'

N = 2000
noise = 0.5

x, t, u_exact, X, T, lb, ub, train_X, train_u, test_X, test_u = preprocess_data_continuous_identification(data_path, N, noise)

train_X = torch.tensor(train_X, dtype = torch.float, requires_grad = True, device = device)
train_u = torch.tensor(train_u, dtype = torch.float, requires_grad = True, device = device)
test_X = torch.tensor(test_X, dtype = torch.float, requires_grad = True, device = device)
test_u = torch.tensor(test_u, dtype = torch.float, requires_grad = True, device = device)

In [5]:
class NeuralNet(nn.Module):

    def __init__(self, layers, lb, ub):
        super().__init__()
        self.lb = torch.tensor(lb, dtype = torch.float, device = device)
        self.ub = torch.tensor(ub, dtype = torch.float, device = device)
        
        # Layer module
        self.layers = nn.ModuleList()
        
        # Make the neural network
        input_dim = layers[0]
        for output_dim in layers[1:]:
            self.layers.append(nn.Linear(input_dim, output_dim))
            nn.init.xavier_normal_(self.layers[-1].weight)
            input_dim = output_dim
        
        
    def forward(self, X):
        x = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for layer in self.layers[:-1]:
            x = torch.tanh(layer(x))

        outputs = self.layers[-1](x)
        return outputs

In [6]:
class PINN:
    def __init__(self, X, u, lb, ub, nu, layers = [2, 20, 20, 20, 1], lr = 1e-2, device = torch.device('cpu')):
        self.nu = nu
        self.x_f = X[:, :1]
        self.t_f = X[:, 1:]
        self.u = u
        
        self.net_u = NeuralNet(layers, lb, ub)
        self.net_u.to(device)
        
        # Physical paramters to optimize
        self.lmbda = nn.ParameterList()
        self.lmbda.append(nn.Parameter(0.0*torch.ones(1, device = device)))
        self.lmbda.append(nn.Parameter(-6.0*torch.ones(1, device = device)))
        
        params = list(self.lmbda) + list(self.net_u.parameters())
        self.optimizer = torch.optim.Adam(params, lr = lr, betas=(0.9, 0.999))
    
    def net_f(self):
        
        u = self.net_u(torch.cat([self.x_f, self.t_f],1))
        
        u_temp1 = torch.ones(self.x_f.size(), device = device)
        u_temp2 = torch.ones(self.x_f.size(), device = device)
        u_temp3 = torch.ones(self.x_f.size(), device = device)
        
        u_t = grad(u, self.t_f, grad_outputs = u_temp1, create_graph = True)[0]
        u_x = grad(u, self.x_f, grad_outputs = u_temp2, create_graph = True)[0]
        u_xx = grad(u_x, self.x_f, grad_outputs = u_temp3, create_graph = True)[0]

        f = u_t + self.lmbda[0]*u*u_x - torch.exp(self.lmbda[1])*u_xx
#         f = u_t + u*u_x - self.nu*u_xx
    
        del u_temp1
        del u_temp2
        del u_temp3
        
        return f
    
    def loss_f(self, u, u_pred):
        return torch.mean(torch.square(u-u_pred)) + torch.mean(torch.square(self.net_f()))
    
    def optimizer_step(self):
        # Zero the grads for the model paramters in the optimizer
        self.optimizer.zero_grad()
            
        # Compute the losses and backpropagate the losses
        loss = self.loss_f(self.u, self.net_u(torch.cat([self.x_f, self.t_f],1)))
        loss.backward()

        # Optimizer one iteration with the given optimizer
        self.optimizer.step()
        
        return loss.item()
    
    def fit(self, epochs = 1):
        for epoch in range(epochs):
            loss_value = self.optimizer_step()

            if epoch % 100 == 99:
                with torch.no_grad():
                    valid_loss = torch.norm(test_u - pinn.predict(test_X), 2)/torch.norm(test_u, 2)
                    print(f'Epoch {epoch+1}: Loss = {loss_value}, Valid Loss = {valid_loss.item()}')
            if epoch % 1000 == 999:
                print(f'Lambda 1 = {self.lmbda[0].item()}, Lambda 2 = {torch.exp(self.lmbda[1]).item()}\n')
                
    def predict(self, test_X):
        return self.net_u(test_X)

In [7]:
# Define the model
pinn = PINN(train_X, train_u, lb, ub, nu = (0.01/np.pi), layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1], lr = 0.0003, device = device)

In [8]:
# Train the model
pinn.fit(epochs = 10000)

Epoch 100: Loss = 0.34559741616249084, Valid Loss = 0.7840022444725037
Epoch 200: Loss = 0.25578761100769043, Valid Loss = 0.621925413608551
Epoch 300: Loss = 0.15509286522865295, Valid Loss = 0.360353022813797
Epoch 400: Loss = 0.14149537682533264, Valid Loss = 0.3170502483844757
Epoch 500: Loss = 0.13986119627952576, Valid Loss = 0.31121739745140076
Epoch 600: Loss = 0.13840632140636444, Valid Loss = 0.3058587312698364
Epoch 700: Loss = 0.1368308663368225, Valid Loss = 0.29983189702033997
Epoch 800: Loss = 0.13507318496704102, Valid Loss = 0.292924702167511
Epoch 900: Loss = 0.13313017785549164, Valid Loss = 0.2850228548049927
Epoch 1000: Loss = 0.1310836374759674, Valid Loss = 0.2762320637702942
Lambda 1 = 0.12217088043689728, Lambda 2 = 0.0030862162820994854

Epoch 1100: Loss = 0.12909330427646637, Valid Loss = 0.2670291066169739
Epoch 1200: Loss = 0.12721791863441467, Valid Loss = 0.25793105363845825
Epoch 1300: Loss = 0.12537342309951782, Valid Loss = 0.2488793134689331
Epoch 140

In [9]:
# Evaluate the model
u_pred = pinn.predict(test_X)
valid_loss = torch.norm(test_u - pinn.predict(test_X), 2)/torch.norm(test_u, 2)
print(valid_loss.item())

0.05325787141919136


In [10]:
# Plot the results
%matplotlib widget
device = torch.device('cpu')

plot_results_continuous_identification(x, t, X, T, u_exact, u_pred.to(device).detach().numpy(),
                                  train_X.to(device).detach().numpy(), train_u.to(device).detach().numpy(),
                                  test_X.to(device).detach().numpy(), pinn.lmbda[0].item(), torch.exp(pinn.lmbda[1]).item())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …